In [2]:
import os
import re
import sys
import time
import warnings
from datetime import datetime

import pymysql
import requests
from tqdm.auto import tqdm

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

from database.conn import AccessDatabase
from crawling.crawler import get_url, json_iterator, get_headers
today = datetime.today().strftime('%y%m%d')

db_glamai = AccessDatabase('glamai')
db_jangho = AccessDatabase('jangho')

warnings.filterwarnings("ignore")

/Users/mycelebs_95/Projects/glamai_data/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### Sephora Product Update

---
#### 1st) Refinement

In [ ]:
from sephora_product.refinement import Refinement
products = Refinement().update_refinement()

---
#### 2nd) Products By Subcategory

In [ ]:
from sephora_product.product_keyword import ProductKeyword
upload_df = ProductKeyword().update_product_keyword()

---
#### 3rd) Update Vertical data

In [ ]:
from sephora_product.vertical_data import VerticalData
VerticalData().update_vertical_data()

---
#### 4th) Update Best & New & Vegan & Organic

In [ ]:
from sephora_product.best_new import UpdateBestSellerNew
from sephora_product.keywords import SephoraVeganOrganic
UpdateBestSellerNew().update_best_new()
SephoraVeganOrganic().update_keywords()

---
#### 5th) Review Date

In [ ]:
from sephora_product.review_date import ReviewDate
new_product_list, data = ReviewDate().update_review_date()

---
#### 6th) Insert product info

In [ ]:
from sephora_product.insert_product_info import update_product_info
result = update_product_info()

---
#### 7th) All product update

In [ ]:
from sephora_product.all_product_update import update_all_product
data = update_all_product()

---
### Search Keywords Update

In [ ]:
from sephora_product.search_keyword import update_search_keywords, db_distinction
total_df = update_search_keywords()
db_distinction()

---
### Sephora Review Update

---
#### 1st) Review Data

In [ ]:
# # backup table
# table = 'sephora_txt_data_re'
# db_glamai._backup(table_name=table, keep=True)

In [ ]:
# update review data
from sephora_review.review_data import ReviewData
txt_data, error = ReviewData()._crawling(backup=True)

In [ ]:
# Check crawling result

columns = ['product_code', 'product_id', 'rating', 'skin_type', 'eye_color', 'skin_concerns', 'hair_color', 'skin_tone', 'age', 'title', 'txt_data', 'positive_count', 'write_time', 'regist_date']
rev_df = pd.DataFrame(txt_data, columns=columns)

error_df = pd.DataFrame(error, columns=['product_code', 'product_url', 'note'])
error_df_cnt = error_df.groupby('note').count()

rev_df.groupby('product_code').count()

print(\
    f"product counts: {len(rev_df.product_code.unique())}\n\
    product review counts: {len(rev_df)}\n\
    reviews that already exist: {error_df_cnt.iloc[0, 0]}\n\
    review does not exist: {error_df_cnt.iloc[1, 0]}"
)

---
#### 2nd) Review Date Update

In [ ]:
from sephora_review.review_data import ReviewDate
result = ReviewDate().update_review_date()

---
#### 3rd) Duplicate check

In [ ]:
'''
/* replace */ 
UPDATE sephora_txt_data_re SET txt_data = REPLACE(txt_data, '.Not impressed.', '.') WHERE BINARY(txt_data) LIKE '%Not impressed.';

/* check duplicated */
select product_code, txt_data, write_time, like_count, count(*) as cnt
from sephora_txt_data_re
group by product_code, txt_data, write_time
having cnt > 1;

/* dedup */

delete t1 
from sephora_txt_data_re t1, sephora_txt_data_re t2 
where 
t1.product_code=t2.product_code and 
t1.txt_data = t2.txt_data and 
t1.write_time =t2.write_time and
t1.like_count < t2.like_count;

delete t1 
from sephora_txt_data_re t1, sephora_txt_data_re t2
where 
t1.product_code = t2.product_code and 
t1.txt_data = t2.txt_data and
t1.write_time = t2.write_time and
t1.like_count = t2.like_count and
t1.pk < t2.pk;
'''

In [ ]:
replace_query = "UPDATE sephora_txt_data_re SET txt_data = REPLACE(txt_data, '.Not impressed.', '.') WHERE BINARY(txt_data) LIKE '%Not impressed.';"
dedup_query_1 = '''
delete t1 
from sephora_txt_data_re t1, sephora_txt_data_re t2 
where 
t1.product_code=t2.product_code and 
t1.txt_data = t2.txt_data and 
t1.write_time =t2.write_time and
t1.like_count < t2.like_count;
'''
dedup_query_2 = '''
delete t1 
from sephora_txt_data_re t1, sephora_txt_data_re t2
where 
t1.product_code = t2.product_code and 
t1.txt_data = t2.txt_data and
t1.write_time = t2.write_time and
t1.like_count = t2.like_count and
t1.pk < t2.pk;'''

conn, curs = db_glamai._connect()
curs.execute(replace_query)
conn.commit()
curs.execute(dedup_query_1)
conn.commit()
curs.execute(dedup_query_2)
conn.commit()
curs.close()
conn.close()

In [ ]:
# Check query

query = '''
select product_code, txt_data, write_time, like_count, count(*) as cnt
from sephora_txt_data_re
group by product_code, txt_data, write_time
having cnt > 1;'''
conn, curs = db_glamai._connect()
curs.execute(query)
data = curs.fetchall()
curs.close()
conn.close()

if len(data) == 0:
    print('Complete dedup!')
else:
    print('Dedup Failed!')

---
### Sephora Product Status 



In [ ]:
# Check status

verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
df_list = []
for vertical in verticals:
    query = f'select is_use, count(*) as count, "{vertical}" as vertical from `sephora_{vertical}_data_status` group by is_use;'
    conn, curs = db_glamai._connect()
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    df_list.append(df)

curs.close()
conn.close()    
status_df = pd.concat(df_list)

In [ ]:
bak_date = input("Enter the date [ ex) `221203` ] :  ")

In [ ]:
# Check product counts
verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
conn, curs = db_glamai._connect()
datas = []
current, paste = 0, 0
for vertical in verticals:
    query = f'''\
    select 'sephora_{vertical}_data_status' as tbl, count(*) as cnt from sephora_{vertical}_data_status union\
    select 'sephora_{vertical}_data_status_bak_{bak_date}', count(*) as cnt from sephora_{vertical}_data_status_bak_{bak_date};'''
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    datas += data
    current += df.iloc[0, 1]
    paste += df.iloc[1, 1]
    
curs.close()
conn.close()    

df = pd.DataFrame(datas)
print(f' - Current: {current}\n - Paste: {paste}\n - Increasement: {current - paste}')

In [ ]:
# Check product counts where is_use=1
verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
conn, curs = db_glamai._connect()
datas = []
current, paste = 0, 0
for vertical in verticals:
    query = f'''\
    select 'sephora_{vertical}_data_status' as tbl, count(*) as cnt from sephora_{vertical}_data_status where is_use=1 union\
    select 'sephora_{vertical}_data_status_bak_{bak_date}', count(*) as cnt from sephora_{vertical}_data_status_bak_{bak_date} where is_use=1;'''
    curs.execute(query)
    data = curs.fetchall()
    df = pd.DataFrame(data)
    print(df, '\n\n')
    datas += data
    current += df.iloc[0, 1]
    paste += df.iloc[1, 1]
    
curs.close()
conn.close()    

df = pd.DataFrame(datas)
print(f' - Current: {current}\n - Paste: {paste}\n - Change: {current - paste}')

---
### Sephora Product Sale

In [ ]:
from sephora_update.sales import update_sephora_sale

price_data_dict = {}
verticals = ['treatments', 'masks', 'eye_care', 'body_care', 'mens', 'fragrance_men', 'fragrance_women', 'wellness', 'cleansers', 'face_base', 'eye', 'lip_color', 'moisturizers', 'cheek']
for vertical in tqdm(verticals):
    price_data = update_sephora_sale(vertical)
    price_data_dict[vertical] = price_data

---
### Affiliate price

In [ ]:
''' glamai.affiliate_price dedup & dup check

/* dedup */
delete 
	t1 
from 
	glamai.affiliate_price t1, glamai.affiliate_price t2
where 
	t1.product_code = t2.product_code and 
	t1.item_no = t2.item_no and
	t1.affiliate_type = t2.affiliate_type and
	t1.update_date < t2.update_date;

/* dup check */	
select product_code, item_no, affiliate_type, count(*) cnt from glamai.affiliate_price group by product_code, item_no, affiliate_type having cnt > 1;

'''

---
#### Amazon update

In [ ]:
from affiliate.amazon import get_data_amazon

In [ ]:
_date = input("Enter the date [ ex) `221203` ] :  ")

tbl = f'affiliate_price_update_amazon_{_date}'
upload_df = db_jangho.get_tbl(tbl)

In [ ]:
upload_df.groupby('is_use').count()

In [ ]:
upload_df[upload_df.is_use==1].groupby('is_sale').count()

In [ ]:
# Check url 
_data = []
for url in upload_df[(upload_df.is_use==-1) | (upload_df.is_use==2)].affiliate_url:
    # wd = get_url(url, window=True, image=True)
    # _data.append(get_data_amazon(url))
    _data.append(get_data_amazon(url, window=True, image=True))
_data

In [ ]:
query = f'''
update glamai.affiliate_price as a
join jangho.affiliate_price_update_amazon_{_date} as b 
on a.product_code = b.product_code and a.item_no = b.item_no and a.affiliate_type = b.affiliate_type
set a.price = b.price, a.sale_price = b.sale_price, a.is_sale = b.is_sale, a.is_use = b.is_use, a.regist_date = b.regist_date, a.update_date = b.update_date
where b.is_use!=-1;'''

conn, curs = db_jangho._connect()
curs.execute(query)
conn.commit()
curs.close()
conn.close()

In [ ]:
# Check query
query = 'select * from affiliate_price where is_use=1 and is_sale=0 and sale_price=0;'
conn, curs = db_glamai._connect()
curs.execute(query)
data = curs.fetchall()
if len(data) == 0:
    print("Complete amazon status data!")
else:
    print("Error: Check status!")
curs.close()
conn.close()

---
#### Ulta update

In [ ]:
_date = input("Enter the date [ ex) `221203` ] :  ")

tbl = f'affiliate_price_update_ulta_{_date}'
upload_df = db_jangho.get_tbl(tbl)
upload_df.groupby('is_use').count()

In [ ]:
upload_df[upload_df.is_use==1].groupby('is_sale').count()

In [ ]:
query = f'''
update glamai.affiliate_price as a
join jangho.affiliate_price_update_ulta_{_date} as b 
on a.product_code = b.product_code and a.item_no = b.item_no and a.affiliate_type = b.affiliate_type
set a.price = b.price, a.sale_price = b.sale_price, a.is_sale = b.is_sale, a.is_use = b.is_use, a.regist_date = b.regist_date, a.update_date = b.update_date
where b.is_use!=-1;'''

conn, curs = db_jangho._connect()
curs.execute(query)
conn.commit()
curs.close()
conn.close()

In [ ]:
# Check query

query = 'select * from affiliate_price where is_use=1 and is_sale=0 and sale_price=0;'
conn, curs = db_glamai._connect()
curs.execute(query)
data = curs.fetchall()
curs.close()
conn.close()

if len(data) == 0:
    print("Complete update ulta sale!")

---
### Walmart update

In [ ]:
import ast
from affiliate._preprocess import preprocess_titles, dup_check, subtractor

#### Preprocessing
---

In [ ]:
'''Glamai Tables
- ds > jangho > `glamai_data`
- ds > jangho > `glamai_detail_data`
'''

columns = ['product_code', 'item_no', 'product_name', 'brand', 'main_vertical', 'size', 'is_use']
glamai_data = db_jangho.get_tbl('glamai_data', columns)
columns = ['product_code', 'item_no', 'color', 'is_use']
glamai_detail = db_jangho.get_tbl('glamai_detail_data', columns)

glamai_df_0 = glamai_data.merge(glamai_detail.loc[:, ['product_code', 'item_no', 'color']], on=['product_code', 'item_no'], how='left')
glamai_df_1 = glamai_data.loc[:, ['product_code', 'product_name', 'brand', 'main_vertical']].merge(glamai_detail.loc[:, ['product_code', 'item_no', 'color', 'is_use']], on=['product_code'], how='left')
glamai_df = pd.concat([glamai_df_0, glamai_df_1]).drop_duplicates(subset=['product_code', 'item_no'], ignore_index=True)

glamai_df.loc[glamai_df['color'].str.strip()=='', 'color'] = None
glamai_df.loc[glamai_df['size']=='', 'size'] = None

'''Walmart Tables
walmart_item_data_{_date}
walmart_variant_data_{_date}
walmart_variant_data_price_{_date}
'''

# /** Enter the Date **/
_date = '221122'
_walmart_df = db_jangho.get_tbl(f'walmart_item_data_{_date}')
_walmart_df.loc[:, 'url'] = 'https://www.walmart.com/' + _walmart_df.loc[:, 'canonicalUrl']

subset = ['usItemId']
walmart_df_dedup = _walmart_df.drop_duplicates(subset=subset, keep='first', ignore_index=True)
col = ['pk', 'usItemId', 'brand', 'product_name']
walmart_df = walmart_df_dedup.loc[:, col]

In [ ]:
preprocessed_df_0 = preprocess_titles(glamai_df)
preprocessed_df_1 = preprocess_titles(walmart_df)
preprocessed_df_0 = preprocessed_df_0[(preprocessed_df_0.brand.notnull()) & (preprocessed_df_0.preprocessed.notnull())].reset_index(drop=True)
preprocessed_df_1 = preprocessed_df_1[(preprocessed_df_1.brand.notnull()) & (preprocessed_df_1.preprocessed.notnull())].reset_index(drop=True)

df_list = []
for idx in tqdm(preprocessed_df_0.index):
    product_code = preprocessed_df_0.loc[idx, 'product_code']
    item_no = preprocessed_df_0.loc[idx, 'item_no']
    
    
    color = preprocessed_df_0.loc[idx, 'color']
    size = preprocessed_df_0.loc[idx, 'size']
    # color_category = preprocessed_df_0.loc[idx, 'color_category']
    title = preprocessed_df_0.loc[idx, 'preprocessed'].lower().replace(' ', '')
    brand = preprocessed_df_0.loc[idx, 'brand'].lower().replace(' ', '')
    
    # 브랜드, 타이틀 모두 일치하는 개체 찾기
    brand_mapped = preprocessed_df_1.brand.str.lower().str.replace(' ', '').str.fullmatch(brand)
    title_mapped = preprocessed_df_1.preprocessed.str.replace(' ', '').str.fullmatch(title)
    mapped_data = preprocessed_df_1[brand_mapped & title_mapped].reset_index(drop=True)
    
    if mapped_data.empty:
        pass
    else:    
        mapped_data.loc[:, 'product_code'] = product_code
        mapped_data.loc[:, 'item_no'] = item_no
        mapped_data.loc[:, 'color'] = color
        mapped_data.loc[:, 'size'] = size
        # mapped_data.loc[:, 'color_category'] = color_category
        df_list.append(mapped_data)

mapped_df = pd.concat(df_list, ignore_index=True)
options_df = db_jangho.get_tbl(f'walmart_variant_data_price_{_date}')
for idx in options_df.index:
    opts = ast.literal_eval(options_df.loc[idx, 'option'])
    _opts = []
    i = 1
    for opt in opts:
        # opt = opt.replace('size-', '').replace('actual_color-', '').strip()
        # options_df.loc[idx, f'option_{i}'] = str(opt)
        # i += 1
        
        if opt[0:12] == 'actual_color':
            color = opt.replace('actual_color-', '').strip()
            options_df.loc[idx, 'walmart_color'] = color
        elif opt[0:4] == 'size':
            size = opt.replace('size-', '').strip()
            options_df.loc[idx, 'walmart_size'] = size
        else:
            options_df.loc[idx, 'another_option'] = opt
            
options_df.info()

In [ ]:
# 대표상품 중 옵션 존재 개체 join
# opt_0 = mapped_df.merge(options_df.loc[:, ['usItemId', 'productId', 'option_1', 'option_2']], on='usItemId', how='left')
opt_0 = mapped_df.merge(options_df.loc[:, ['usItemId', 'productId', 'walmart_color', 'walmart_size', 'another_option']], on='usItemId', how='left')

# 매핑 상품에 옵션 부여
pks = mapped_df.pk.unique() # 매핑 완료 개체 (대표상품)
ids = opt_0[opt_0.productId.notnull()].usItemId.unique() # 대표상품 중 옵션 존재 usItemId
# col = ['pk', 'productId', 'usItemId', 'product_name', 'option_1', 'option_2']
col = ['pk', 'productId', 'usItemId', 'product_name', 'walmart_color', 'walmart_size', 'another_option']
_opt_1 = options_df.loc[(options_df.pk.isin(pks)) & (options_df.usItemId.isin(ids)==False), col]

col = ['pk', 'product_code', 'item_no', 'brand', 'color', 'size']
_mapped_df = mapped_df.loc[:, col]
opt_1 = _opt_1.merge(_mapped_df, on='pk', how='left')

# concat
mapped_opt_df = pd.concat([opt_0, opt_1]).sort_values(by=['pk', 'usItemId'], ignore_index=True)

# convert np.nan to None
mapped_opt_df = mapped_opt_df.where(pd.notnull(mapped_opt_df), None)

mapped_opt_df.info()

In [ ]:
# cols = ['option_1', 'option_2', 'volume_oz', 'volume_ml', 'volume_kg']
cols = ['walmart_color', 'walmart_size', 'another_option', 'volume_oz', 'volume_ml', 'volume_kg']
opts = ['color', 'size']
# opts = ['color']
for idx in mapped_opt_df.index:
    if mapped_opt_df.loc[idx, cols+opts].isnull().values.tolist().count(True) == len(opts) + len(cols):
        # 옵션 자체가 존재하지 않음
        status = -1       
        attrs = None
    else:
        attrs = []
        for opt in opts:    
            opt = mapped_opt_df.loc[idx, opt]
            if opt is None:
                pass
            else:
                opt = opt.lower().replace(' ', '')
                col_cnt = 0
                for col in cols:
                    attr = mapped_opt_df.loc[idx, col]
                    if attr is None:
                        pass
                    else:    
                        attr = attr.lower().replace(' ', '')
                        if attr in opt:
                            attrs.append(attr)
        if len(attrs) == 0:
            # 옵션은 존재하지만 일치하지 않음
            status = 0
            attrs = None
        else:
            # 옵션 일치
            status = 1
            attrs = str(list(set(attrs))) 
    
    mapped_opt_df.loc[idx, 'attributes'] = attrs
    mapped_opt_df.loc[idx, 'mapped_status'] = status

# group by mapped_status count
mapped_opt_df.groupby('mapped_status').count()

In [ ]:
columns = ['product_code', 'item_no', 'brand', 'pk', 'usItemId', 'productId', 'attributes']
mapped_df_comp = mapped_opt_df.loc[mapped_opt_df.mapped_status==1, columns].sort_values(by=['product_code', 'item_no'], ignore_index=True)
print(f'- Glamai data: {len(glamai_df)}\n- Walmart data: {len(walmart_df)}\n- Mapping data: {len(mapped_df_comp)}')

In [ ]:
# Test Table
# db_jangho.engine_upload(mapped_opt_df, f'sephora_to_walmart_mapped_test_{_date}', if_exists_option='replace')

# Table Upload: ds > jangho > sephora_to_walmart_mapped_{_date}
db_jangho.engine_upload(mapped_opt_df, f'sephora_to_walmart_mapped_{_date}', if_exists_option='replace')

# mapped_opt_df = db_jangho.get_tbl(f'sephora_to_walmart_mapped_{_date}')

#### Maaping: sephora to walmart
---

In [ ]:
item_df = db_jangho.get_tbl(f'walmart_item_data_{_date}')
v_p_df = db_jangho.get_tbl(f'walmart_variant_data_price_{_date}')
map_df = db_jangho.get_tbl(f'sephora_to_walmart_mapped_{_date}')
cols = ['product_code', 'item_no', 'usItemId', 'mapped_status']
_map_df = map_df.loc[:, cols]

cols = ['usItemId', 'canonicalUrl', 'price', 'sale_price', 'availabilityStatus']
_item_df = item_df.loc[:, cols]
_v_p_df = v_p_df.loc[:, cols]

In [ ]:
# Merge: price data
mer_df_0 = _map_df.merge(_item_df, on='usItemId', how='inner')
mer_df_1 = _map_df.merge(_v_p_df, on='usItemId', how='inner')

mer_df = pd.concat([mer_df_0, mer_df_1], ignore_index=True)
subset = ['product_code', 'item_no', 'usItemId']
dedup_df = mer_df.drop_duplicates(subset=subset, keep='last', ignore_index=True)

# affiliate data
dedup_df.loc[:, 'affiliate_type'] = 'walmart'
dedup_df.loc[:, 'affiliate_url'] = 'https://www.walmart.com' + dedup_df['canonicalUrl']
dedup_df.loc[:, 'affiliate_image'] = 'https://alls3.glamai.com/images/affiliate/walmart.jpg'
# stock status check (is_use)
dedup_df.loc[(dedup_df['availabilityStatus']=='In stock') | (dedup_df['availabilityStatus']=='IN_STOCK'), 'is_use'] = True
dedup_df.loc[(dedup_df['availabilityStatus']!='In stock') & (dedup_df['availabilityStatus']!='IN_STOCK'), 'is_use'] = False

# sale status check (is_sale)
dedup_df.loc[dedup_df['price']>dedup_df['sale_price'], 'is_sale'] = True
dedup_df.loc[dedup_df['price']==dedup_df['sale_price'], 'is_sale'] = False

dedup_df.loc[dedup_df['sale_price']==0, 'is_sale'] = False
dedup_df.loc[dedup_df['sale_price']==0, 'is_use'] = False
columns = ['product_code', 'item_no', 'affiliate_type', 'affiliate_url', 'affiliate_image', 'usItemId', 'price', 'sale_price', 'is_sale', 'is_use']
upload_df = dedup_df.loc[dedup_df.mapped_status==1, columns].sort_values(by=['product_code', 'item_no'], ignore_index=True)
upload_df.loc[:, 'regist_date'] = pd.Timestamp(datetime.today().strftime("%Y-%m-%d"))
upload_df.loc[:, 'update_date'] = pd.Timestamp(datetime.today().strftime("%Y-%m-%d"))

upload_df.info()
upload_df[upload_df.is_use==False]

In [ ]:
"""
월마트 내부 중복 체크 
 - dedup = 1(True): 대표상품 (가격 낮은 상품)
 - dedup = 0(False): 종속상품 (가격 높은 상품 or is_use = 0(False))
"""
_upload_df = upload_df[upload_df.is_use].sort_values(by='sale_price', ignore_index=True)
upload_df_ = upload_df[upload_df.is_use==False]

subset = ['product_code', 'item_no']
dedup_df = _upload_df.drop_duplicates(subset=subset, keep='first', ignore_index=True)
dup_df = dup_check(df=_upload_df, subset=subset, keep='first', sorting=True)

dedup_df.loc[:, 'dedup'] = True
if not upload_df_.empty:
    upload_df_ = upload_df_.reset_index(drop=True)
    upload_df_.loc[:, 'dedup'] = False # is_use=0(False) 이면 중복으로 간주 -> 서비스 테이블에 insert 안 되게 하기 위함
dup_df.loc[:, 'dedup'] = False
concat_df = pd.concat([dedup_df, upload_df_, dup_df]).sort_values(by=subset, ignore_index=True)

concat_df.info()

In [ ]:
# concat_df.groupby('is_use').count()
concat_df.groupby('dedup').count()

In [ ]:
# Table Upload: ds > jangho > sephora_to_walmart_mapped
db_jangho.create_table(upload_df=concat_df, table_name='sephora_to_walmart_mapped')

In [ ]:
query = f'''
update glamai.affiliate_price as a
join jangho.sephora_to_walmart_mapped as b 
on a.product_code = b.product_code and a.item_no = b.item_no and a.affiliate_type = b.affiliate_type
set a.price = b.price, a.sale_price = b.sale_price, a.is_sale = b.is_sale, a.is_use = b.is_use, a.update_date = b.update_date
where b.dedup=1;'''

data = db_jangho._execute(query)
print(data)

In [ ]:
query = f"""
SELECT product_code, item_no
FROM affiliate_price as a
WHERE a.is_use=1 and a.affiliate_type='walmart';
"""

data = db_glamai._execute(query)
df = pd.DataFrame(data)

In [ ]:
dedup_df = concat_df.loc[concat_df.dedup, ['product_code', 'item_no', 'affiliate_type', 'affiliate_url', 'affiliate_image', 'price', 'sale_price', 'is_sale', 'is_use', 'regist_date', 'update_date']]

subset=['product_code', 'item_no']
new_df = subtractor(dedup_df, df, subset)
print(f"\n\nNew product counts: {len(new_df)}\n\n")

In [ ]:
affi_tbl = "affiliate_price"

# Table bakcup: ds > glamai > affiliate_price_bak_{_date}
db_glamai._backup(table_name=affi_tbl, keep=True)

# Table upload: ds > glamai > affiliate_price
db_glamai.engine_upload(upload_df=new_df, table_name=affi_tbl, if_exists_option='append')

In [ ]:
# glamai.affiliate_price dedup & dup check

# /* dedup */

dedup_query = f"""
delete 
	t1 
from 
	glamai.affiliate_price t1, glamai.affiliate_price t2
where 
	t1.product_code = t2.product_code and 
	t1.item_no = t2.item_no and
	t1.affiliate_type = t2.affiliate_type and
	t1.update_date < t2.update_date;"""

# /* dup check */	

dup_check_query = f"""
select product_code, item_no, affiliate_type, count(*) cnt from glamai.affiliate_price group by product_code, item_no, affiliate_type having cnt > 1;
"""


In [ ]:
data = db_glamai._execute(dup_check_query)
df = pd.DataFrame(data)
if df.empty:
    print("중복 제거 완료.")
else:
    print("중복인 행이 존재합니다. 중복제거 진행합니다.")
    data = db_glamai._execute(dedup_query)